## Sci-Fi Story Generator using RNN (LSTM)
### Final Project - NLP for Creatives (Peckham DAZ Programme 2024/25)
#### 🔍 Project Overview

This project explores how Recurrent Neural Networks (RNNs), specifically Long Short-Term Memory (LSTM) networks, can be used to generate sci-fi stories. A dataset of public domain science fiction texts is used to train a language model that learns how to predict the next word in a sequence and generate creative story snippets.

In [ ]:
!pip install nltk tensorflow

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
import re
import requests

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/huggingface_hub-0.29.2-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/screenshareuk/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Load from Project Gutenberg (sci-fi book)
url = "http://www.gutenberg.org/cache/epub/2147/pg2147.txt"
response = requests.get(url)
text = response.text

# Basic cleaning
text = text.lower()
text = re.sub(r'[^a-z\s]', '', text)
nltk.download('punkt')
print(nltk.data.path)
tokens = word_tokenize(text)

print("Total tokens:", len(tokens))
print("First 50 tokens:", tokens[:50])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/screenshareuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['/Users/screenshareuk/nltk_data', '/opt/anaconda3/nltk_data', '/opt/anaconda3/share/nltk_data', '/opt/anaconda3/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
Total tokens: 93144
First 50 tokens: ['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'works', 'of', 'edgar', 'allan', 'poe', 'volume', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse']


In [ ]:
# Create tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)
total_words = len(tokenizer.word_index) + 1

# Create sequences
input_sequences = []
seq_length = 50

for i in range(seq_length, len(tokens)):
    seq = tokens[i-seq_length:i+1]
    encoded = tokenizer.texts_to_sequences([seq])[0]
    input_sequences.append(encoded)

# Pad and split into X and y
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length+1))
X = input_sequences[:, :-1]
y = to_categorical(input_sequences[:, -1], num_classes=total_words)

print("Vocabulary size:", total_words)
print("Input shape:", X.shape)

Vocabulary size: 10388
Input shape: (93094, 50)


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, verbose=1)

Epoch 1/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 237s 320ms/step - accuracy: 0.0779 - loss: 7.2126
Epoch 2/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 268s 368ms/step - accuracy: 0.1036 - loss: 6.3671
Epoch 3/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 261s 358ms/step - accuracy: 0.1153 - loss: 6.1106
Epoch 4/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 270s 371ms/step - accuracy: 0.1281 - loss: 5.9033
Epoch 5/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 275s 378ms/step - accuracy: 0.1366 - loss: 5.7449
Epoch 6/20
 49/728 ━━━━━━━━━━━━━━━━━━━━ 4:41 414ms/step - accuracy: 0.1405 - loss: 5.6544

KeyboardInterrupt: 

In [ ]:
⚠️ Note: Training might take a while, especially without GPU.


In [ ]:
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted)
        output_word = tokenizer.index_word.get(predicted_index, '')
        seed_text += " " + output_word
    return seed_text

In [ ]:
seed = "in the year 3000 a group of astronauts discovered"
story = generate_text(seed, next_words=50)
print("✨ Generated Sci-Fi Text:\n", story)

#### 🧠 Reflections and Ethical Notes

- **Learning experience:** I learned how to build and train a language model using RNNs.
- **Challenges:** Memory usage and training time were the main challenges. Using Google Colab helped.
- **Limitations:** The model sometimes repeats phrases or generates grammatically incorrect text.
- **Ethics:** I used public domain data. The model is intended for creative storytelling, not factual information.
- **LLM Disclaimer:** I used ChatGPT to assist in structuring this project, coding, and documentation. All code was tested and understood by me.
